In [ ]:
!apt-get install -y wget gnupg
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | apt-key add -
!echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!apt-get update
!apt-get install -y mongodb-database-tools
!pip install pymongo

In [ ]:
from pymongo import MongoClient, UpdateOne
from collections import deque

client = MongoClient("")

# Choose your database and collection
db = client["testdb"]          # <-- replace with your DB name
collection = db["sensors"]     # <-- replace with your collection name

In [ ]:
# # --- Config ---
# CHUNK_FETCH = 5000   # fetch this many docs at once
# PROCESS_SIZE = 4950  # process only 4900 docs (leave overlap)
# BULK_LIMIT = 1000    # flush bulk writes every 1000 updates

# bulk_updates = []
# carryover = []   # keep last 2 docs from previous batch
# count = 0

# def flush_updates():
#     """Write accumulated bulk updates to DB."""
#     global bulk_updates
#     if bulk_updates:
#         collection.bulk_write(bulk_updates, ordered=False)
#         bulk_updates = []

# # --- Stream through DB in batches ---
# batch_cursor = collection.find({}, {"Server Timestamp": 1, "Value": 1})
# batch = []

# for doc in batch_cursor:
#     batch.append(doc)
#     if len(batch) == CHUNK_FETCH:
#         # merge with carryover from previous batch
#         chunk = carryover + batch

#         # sort in memory by timestamp
#         chunk = sorted(chunk, key=lambda x: x["Server Timestamp"])

#         # keep last 2 docs for overlap to next chunk
#         carryover = chunk[-2:]
#         process_chunk = chunk[:-2]   # leave last 2 aside

#         # process this chunk
#         window = []
#         for curr_doc in process_chunk:
#             window.append(curr_doc)
#             if len(window) < 3:
#                 continue

#             prev, curr, nxt = window[0], window[1], window[2]
#             anomaly_flag = None
#             new_value = None

#             # ----------- RESET -----------
#             if curr["Value"] < prev["Value"] and nxt["Value"] > prev["Value"]:
#                 anomaly_flag = "Yes (Reset)"

#             # ----------- FAULTY SENSOR -----------
#             elif curr["Value"] == 0:
#                 zero_block = [curr]
#                 while len(window) >= 3 and window[1]["Value"] == 0:
#                     zero_block.append(window[1])
#                     window.pop(1)

#                 if len(window) >= 3:
#                     after_val = window[1]["Value"]
#                     if after_val < prev["Value"]:
#                         new_value = (prev["Value"] + after_val) / 2
#                         for z in zero_block:
#                             bulk_updates.append(UpdateOne(
#                                 {"_id": z["_id"]},
#                                 {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
#                             ))
#                     else:
#                         bulk_updates.append(UpdateOne(
#                             {"_id": zero_block[0]["_id"]},
#                             {"$set": {"Anomaly Flag": "Yes (Reset)"}}
#                         ))
#                 else:  # dataset finished → copy prev
#                     new_value = prev["Value"]
#                     for z in zero_block:
#                         bulk_updates.append(UpdateOne(
#                             {"_id": z["_id"]},
#                             {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
#                         ))
#                 continue

#             # ----------- NEGATIVE DIFFERENCE -----------
#             elif nxt["Value"] > curr["Value"]:
#                 anomaly_flag = "Yes (Negative Difference)"

#             # --- Add to bulk ---
#             update_fields = {}
#             if anomaly_flag:
#                 update_fields["Anomaly Flag"] = anomaly_flag
#             if new_value is not None:
#                 update_fields["Value"] = new_value

#             if update_fields:
#                 bulk_updates.append(UpdateOne({"_id": curr["_id"]}, {"$set": update_fields}))

#             # --- Flush bulk if needed ---
#             if len(bulk_updates) >= BULK_LIMIT:
#                 flush_updates()

#             window.pop(0)
#             count += 1

#         # reset batch
#         batch = []

# # flush any remaining
# flush_updates()
# print(f"Finished processing {count} rows.")

In [ ]:
#SCRIPT TO PROCESS THE PREVIOULY COLLECTED LARGE DATA (OVER 1M+ RECORDS)

from pymongo import UpdateOne

# --- Config ---
CHUNK_FETCH = 5000   # fetch this many docs at once
PROCESS_SIZE = 4950  # process only 4900 docs (leave overlap)
BULK_LIMIT = 1000    # flush bulk writes every 1000 updates

bulk_updates = []
carryover = []   # keep last 2 docs from previous batch
count = 0

def flush_updates():
    """Write accumulated bulk updates to DB."""
    global bulk_updates
    if bulk_updates:
        collection.bulk_write(bulk_updates, ordered=False)
        bulk_updates = []

# --- Stream through DB in batches ---
batch_cursor = collection.find({}, {"Server Timestamp": 1, "Value": 1})
batch = []

for doc in batch_cursor:
    batch.append(doc)
    if len(batch) == CHUNK_FETCH:
        # merge with carryover from previous batch
        chunk = carryover + batch

        # sort in memory by timestamp
        chunk = sorted(chunk, key=lambda x: x["Server Timestamp"])

        # keep last 2 docs for overlap to next chunk
        carryover = chunk[-2:]
        process_chunk = chunk[:-2]   # leave last 2 aside

        # process this chunk
        window = []
        for curr_doc in process_chunk:
            window.append(curr_doc)
            if len(window) < 3:
                continue

            prev, curr, nxt = window[0], window[1], window[2]
            anomaly_flag = None
            new_value = None

            # ----------- RESET -----------
            if curr["Value"] < prev["Value"] and nxt["Value"] > prev["Value"]:
                anomaly_flag = "Yes (Reset)"

            # ----------- FAULTY SENSOR -----------
            elif curr["Value"] == 0:
                zero_block = [curr]
                while len(window) >= 3 and window[1]["Value"] == 0:
                    zero_block.append(window[1])
                    window.pop(1)

                if len(window) >= 3:
                    after_val = window[1]["Value"]
                    if after_val < prev["Value"]:
                        new_value = (prev["Value"] + after_val) / 2
                        for z in zero_block:
                            bulk_updates.append(UpdateOne(
                                {"_id": z["_id"]},
                                {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
                            ))
                    else:
                        bulk_updates.append(UpdateOne(
                            {"_id": zero_block[0]["_id"]},
                            {"$set": {"Anomaly Flag": "Yes (Reset)"}}
                        ))
                else:  # dataset finished → copy prev
                    new_value = prev["Value"]
                    for z in zero_block:
                        bulk_updates.append(UpdateOne(
                            {"_id": z["_id"]},
                            {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
                        ))
                continue

            # ----------- NEGATIVE DIFFERENCE -----------
            elif nxt["Value"] > curr["Value"]:
                anomaly_flag = "Yes (Negative Difference)"

            # --- Add to bulk ---
            update_fields = {}
            if anomaly_flag:
                update_fields["Anomaly Flag"] = anomaly_flag
            if new_value is not None:
                update_fields["Value"] = new_value

            if update_fields:
                bulk_updates.append(UpdateOne({"_id": curr["_id"]}, {"$set": update_fields}))

            # --- Flush bulk if needed ---
            if len(bulk_updates) >= BULK_LIMIT:
                flush_updates()

            window.pop(0)
            count += 1

            # --- NEW: Log every 50k processed rows ---
            if count % 50000 == 0:
                print(f"Processed {count} rows...")

        # reset batch
        batch = []

# flush any remaining
flush_updates()
print(f"Finished processing {count} rows.")


In [ ]:
#SCRIPT FOR SCHEDULER TO PROCESS LAST HOUR RECORDS WITH THE LOGGING INFO

import time
from datetime import datetime, timedelta
from pymongo import MongoClient, UpdateOne

# --- Config ---
BUFFER_SIZE = 10       # keep last N records as buffer (editable)
PROCESS_SIZE = 60      # process this many records per run (editable)
BULK_LIMIT = 1000      # flush bulk writes every 1000 updates

def flush_updates(bulk_updates):
    """Write accumulated bulk updates to DB."""
    if bulk_updates:
        collection.bulk_write(bulk_updates, ordered=False)
        return len(bulk_updates)
    return 0

def process_last_hour():
    start_time = time.time()
    now = datetime.utcnow()
    one_hour_ago = now - timedelta(hours=1)

    # Fetch last 1 hour records (sorted by timestamp)
    records = list(collection.find(
        {"Server Timestamp": {"$gte": one_hour_ago, "$lt": now}},
        {"Server Timestamp": 1, "Value": 1}
    ).sort("Server Timestamp", 1))

    total_records = len(records)

    if total_records <= BUFFER_SIZE:
        print(f"[{now}] Not enough records ({total_records}) to process. Skipping.")
        return

    # Keep buffer, process rest
    process_records = records[:-BUFFER_SIZE]
    process_records = process_records[:PROCESS_SIZE]   # only process up to PROCESS_SIZE
    bulk_updates = []
    count = 0
    modifications = 0
    window = []

    for curr_doc in process_records:
        window.append(curr_doc)
        if len(window) < 3:
            continue

        prev, curr, nxt = window[0], window[1], window[2]
        anomaly_flag = None
        new_value = None

        # ----------- RESET -----------
        if curr["Value"] < prev["Value"] and nxt["Value"] > prev["Value"]:
            anomaly_flag = "Yes (Reset)"

        # ----------- FAULTY SENSOR -----------
        elif curr["Value"] == 0:
            zero_block = [curr]
            while len(window) >= 3 and window[1]["Value"] == 0:
                zero_block.append(window[1])
                window.pop(1)

            if len(window) >= 3:
                after_val = window[1]["Value"]
                if after_val < prev["Value"]:
                    new_value = (prev["Value"] + after_val) / 2
                    for z in zero_block:
                        bulk_updates.append(UpdateOne(
                            {"_id": z["_id"]},
                            {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
                        ))
                        modifications += 1
                else:
                    bulk_updates.append(UpdateOne(
                        {"_id": zero_block[0]["_id"]},
                        {"$set": {"Anomaly Flag": "Yes (Reset)"}}
                    ))
                    modifications += 1
            else:  # dataset finished → copy prev
                new_value = prev["Value"]
                for z in zero_block:
                    bulk_updates.append(UpdateOne(
                        {"_id": z["_id"]},
                        {"$set": {"Anomaly Flag": "Yes (Faulty)", "Value": new_value}}
                    ))
                    modifications += 1
            continue

        # ----------- NEGATIVE DIFFERENCE -----------
        elif nxt["Value"] > curr["Value"]:
            anomaly_flag = "Yes (Negative Difference)"

        # --- Add to bulk ---
        update_fields = {}
        if anomaly_flag:
            update_fields["Anomaly Flag"] = anomaly_flag
        if new_value is not None:
            update_fields["Value"] = new_value

        if update_fields:
            bulk_updates.append(UpdateOne({"_id": curr["_id"]}, {"$set": update_fields}))
            modifications += 1

        # --- Flush bulk if needed ---
        if len(bulk_updates) >= BULK_LIMIT:
            modifications += flush_updates(bulk_updates)
            bulk_updates = []

        window.pop(0)
        count += 1

    # Flush remaining
    modifications += flush_updates(bulk_updates)

    # --- Logging ---
    end_time = time.time()
    duration = round(end_time - start_time, 2)
    print(f"""
========== HOURLY PROCESS REPORT ==========
Time Window   : {one_hour_ago} → {now}
Records Found : {total_records}
Processed     : {count}
Modifications : {modifications}
Time Taken    : {duration} seconds
===========================================
""")

# --- Run Once (Scheduler will call this every hour) ---
if __name__ == "__main__":
    process_last_hour()
